# 模型与层

在Keras中有两个非常重要的概念：模型和层。

## 1. 自定义层

一个模型中通常包含多个"层"，且层与层之间有很多复杂的操作，所以自定义层就是你属否熟悉tensorflow的一个里程碑。

在tensorflow2自定义层非常简单，只需要继承`tf.keras.`

In [3]:
import tensorflow as tf

class CustomLinearLayer(tf.keras.layers.Layer):
    def __init__(self, units: int):
        super().__init__()
        self.units = units

    def build(self, input_shape: tf.Tensor):
        """build layer on the data flow through the model"""

        self.w_weight = self.add_weight(
            shape=(input_shape.shape[-1], self.units),
            initializer=tf.keras.initializers.glorot_normal(),
        )

        self.b_weight = self.add_weight(
            shape=(input_shape.shape[-1]),
            initializer=tf.keras.initializers.glorot_normal()
        )

    def call(self, inputs: tf.Tensor):
        """forward the dataset"""
        # w * input + b
        output = tf.add(tf.matmul(self.w_weight, inputs), tf.expand_dims(self.b_weight, axis=-1))
        return output

(2, 3, 1)


在自定义层的过程中需要注意几点：
- 自定义层是需要继承`tf.keras.layers.Layer`的，然后充血制定的函数即可。
- build 函数是在真实数据第一次流经该层的时候才会执行的，且只会执行一次。作用在于能够动态的根据真实数据的大小创建层。如果大家从pytorch转过来的同学们可能不会习惯，可是如果熟悉功能后会发现是真的好用。
- 在非init函数中添加需要学习的参数时，是需要使用`add_weight`方法来添加相关参数的，不然在最后通过`variables`来获取该层的参数时是无法获取到的。

## 2. 自定义模型

In [7]:
class CustomA(tf.keras.Model):

    def __init__(self):
        print("customA init()")
        super().__init__()
        self.dense = tf.keras.layers.Dense(20)

    def get_config(self):
        return super().get_config()

    def build(self, input_shape):
        print("customA build()")

    def call(self, inputs, training=None, mask=None):
        print("customA call()")
        return self.dense(inputs)


class CustomB(tf.keras.Model):

    def __init__(self):
        print("customB init()")
        super().__init__()
        self.dense = tf.keras.layers.Dense(10)

    def get_config(self):
        return super().get_config()

    def build(self, input_shape):
        print("customB build()")

    def call(self, inputs, training=None, mask=None):
        print("customB call()")
        return self.dense(inputs)

class MyModel(tf.keras.Model):
    def __init__(self):
        print("MyModel init()")
        super().__init__()
        self.a = CustomA()
        self.b = CustomB()

    def build(self, input_shape):
        print("MyModel build()")

    def get_config(self):
        return super().get_config()

    def call(self, inputs, training=None, mask=None):
        print("MyModel call()")
        output = self.a(inputs)
        output = self.b(output)
        return output


inputs = tf.random.uniform((100,45))
model = MyModel()
outputs = model(inputs)
print(outputs.shape)

MyModel init()
customA init()
customB init()
MyModel build()
MyModel call()
customA build()
customA call()
customB build()
customB call()
(100, 10)


## 3. 例子：全连接模型

In [10]:
from tqdm.notebook import tqdm
import tensorflow as tf

x = tf.random.uniform(shape=(2, 3))
y = tf.random.uniform(shape=(2, 1))

class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense = tf.keras.layers.Dense(
            units=1,
            activation=None,
            kernel_initializer=tf.keras.initializers.GlorotNormal(),
            bias_initializer=tf.keras.initializers.GlorotNormal()
        )
    def call(self, input: tf.Tensor):
        """forward the trainning data on the model"""
        output = self.dense(input)
        return output

model = Linear()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
epochs = 1000

# 需要学习的参数

with tqdm(total=epochs) as bar:
    for i in range(epochs):
        with tf.GradientTape() as tape:
            y_hat = model(x)
            loss = tf.reduce_sum(tf.square(y_hat, y))
        
        bar.update()
        bar.set_description(f'loss: {loss}')
        # 这里不能将所有变量提取出来，不然会导致损失值无法得到更新。
        # 因为tensorflow的模型拥有延迟构建的作用
        gradients = tape.gradient(loss, model.variables)
        optimizer.apply_gradients(grads_and_vars=zip(gradients, model.variables))